# Stacked LSTMs for Time Series Regression

### Loading Libraries

In [16]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
import pandas_datareader.data as web

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Warning
import warnings

# Path
from pathlib import Path

# SciPy
from scipy.stats import spearmanr

# Scikit-Learn
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# TensorFlow
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Input, concatenate, Embedding, Reshape, BatchNormalization

In [17]:
np.random.seed(42)

idx = pd.IndexSlice

sns.set_style('whitegrid')

In [18]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')

Using GPU


In [19]:
results_path = Path('results', 'lstm_embeddings')

if not results_path.exists():
    results_path.mkdir(parents=True)

### Data

In [25]:
data = pd.read_hdf('data.h5', 'returns_weekly').drop('label', axis=1)

In [27]:
data['ticker'] = pd.factorize(data.index.get_level_values('ticker'))[0]

In [29]:
data['month'] = data.index.get_level_values('date').month

data = pd.get_dummies(data, columns=['month'], prefix='month')

In [31]:
data.info()

### Train-Test Split

In [34]:
window_size=52

sequence = list(range(1, window_size+1))

ticker = 1

months = 12

n_tickers = data.ticker.nunique()

In [54]:
train_data = data.loc[idx[:, :'2016'], :]

test_data = data.loc[idx[:, '2017'],:]

In [56]:
X_train = [
    train_data.loc[:, sequence].values.reshape(-1, window_size , 1),
    train_data.ticker,
    train_data.filter(like='month')
]

y_train = train_data.fwd_returns

[x.shape for x in X_train], y_train.shape

In [58]:
X_test = [
    test_data.loc[:, list(range(1, window_size+1))].values.reshape(-1, window_size , 1),
    test_data.ticker,
    test_data.filter(like='month')
]

y_test = test_data.fwd_returns

[x.shape for x in X_test], y_test.shape

### Defining Model Architecture

In [60]:
K.clear_session()

In [62]:
n_features = 1

In [64]:
returns = Input(shape=(window_size, n_features), name='Returns')

tickers = Input(shape=(1,), name='Tickers')

months = Input(shape=(12,), name='Months')

### LSTM Layers

In [68]:
lstm1_units = 25

lstm2_units = 10

In [70]:
lstm1 = LSTM(units=lstm1_units, 
             input_shape=(window_size, 
                          n_features), 
             name='LSTM1', 
             dropout=.2,
             return_sequences=True)(returns)

lstm_model = LSTM(units=lstm2_units, 
             dropout=.2,
             name='LSTM2')(lstm1)

#### Embedding Layer

In [73]:
ticker_embedding = Embedding(input_dim=n_tickers, 
                             output_dim=5, 
                             input_length=1)(tickers)

ticker_embedding = Reshape(target_shape=(5,))(ticker_embedding)

#### Concatenating Model Components

In [76]:
merged = concatenate([lstm_model, 
                      ticker_embedding, 
                      months], name='Merged')

bn = BatchNormalization()(merged)
hidden_dense = Dense(10, name='FC1')(bn)

output = Dense(1, name='Output')(hidden_dense)

rnn = Model(inputs=[returns, tickers, months], outputs=output)

In [78]:
rnn.summary()

### Training Model

In [97]:
optimizer =tf.keras.optimizers.Adam()

rnn.compile(loss='mse',
            optimizer=optimizer)

In [99]:
lstm_path = (results_path / 'lstm.regression.h5').as_posix()

checkpointer = ModelCheckpoint(filepath=lstm_path,
                               verbose=1,
                               monitor='val_loss',
                               mode='min',
                               save_best_only=True)

In [101]:
early_stopping = EarlyStopping(monitor='val_loss', 
                              patience=5,
                              restore_best_weights=True)

In [103]:
training = rnn.fit(X_train,
                   y_train,
                   epochs=50,
                   batch_size=64,
                   validation_data=(X_test, y_test),
                   callbacks=[early_stopping, checkpointer],
                   verbose=1)

In [105]:
loss_history = pd.DataFrame(training.history)

#### Evaluating Model Performance

In [108]:
test_predict = pd.Series(rnn.predict(X_test).squeeze(), index=y_test.index)

In [110]:
df = y_test.to_frame('ret').assign(y_pred=test_predict)

In [112]:
by_date = df.groupby(level='date')

df['deciles'] = by_date.y_pred.apply(pd.qcut, q=5, labels=False, duplicates='drop')

In [114]:
ic = by_date.apply(lambda x: spearmanr(x.ret, x.y_pred)[0]).mul(100)

In [116]:
df.info()

In [118]:
test_predict = test_predict.to_frame('prediction')

test_predict.index.names = ['symbol', 'date']

test_predict.to_hdf(results_path / 'predictions.h5', 'predictions')

In [120]:
rho, p = spearmanr(df.ret, df.y_pred)

print(f'{rho*100:.2f} ({p:.2%})')

In [122]:
fig, axes = plt.subplots(ncols=2, figsize=(14,4))

sns.barplot(x='deciles', y='ret', data=df, ax=axes[0])

axes[0].set_title('Weekly Fwd Returns by Predicted Quintile')
axes[0].yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.2%}'.format(y))) 
axes[0].set_ylabel('Weekly Returns')
axes[0].set_xlabel('Quintiles')

avg_ic = ic.mean()
title = f'4-Week Rolling IC | Weekly avg: {avg_ic:.2f} | Overall: {rho*100:.2f}'
ic.rolling(4).mean().dropna().plot(ax=axes[1], title=title)
axes[1].axhline(avg_ic, ls='--', c='k', lw=1)
axes[1].axhline(0, c='k', lw=1)
axes[1].set_ylabel('IC')
axes[1].set_xlabel('Date')

sns.despine()
fig.tight_layout()
fig.savefig(results_path / 'lstm_reg');
plt.grid()
plt.show()